In [3]:
import numpy as np
import random

#import h5py.highlevel
import astropy.io.fits as fits
from astropy.table import Table

import pandas as pd
import seaborn as sns
sns.set(style="ticks", context="talk", font_scale=1.3)

import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
## get data
def _getData(filename, lower=3400, upper=8200):
    obj=fits.open(filename)    
    data = Table.read(obj, hdu=1).to_pandas()
    z = Table.read(obj,hdu=2)['Z']
    print('min_lam =', np.power(10,min(data['loglam'])),'  max_lam=',np.power(10,max(data['loglam'])))
    data['lam'] = np.power(10,data['loglam'])/(1+z)
    data = data[(data['lam']>lower) & (data['lam']<upper)]
    obj.close()
    return data


def normalEquation(x,y,smoothness,xs):
    # weight as diagnol matrix
    w = np.diag(weight(x,xs,smoothness))
    # compute theta using normal equation
    # theta=(x'wx)x′wy
    left = np.linalg.pinv(np.dot(np.transpose(x),np.dot(w,x)))
    right = np.dot(np.transpose(x),np.dot(w,y))
    theta = np.dot(left,right)
    return theta

def weight(x,xs,smoothness):
    # distance and its square
    D = x - xs
    DSquare = np.diagonal(np.dot(D,np.transpose(D)))    
    # select normalization factor for a given smoothness
    position = int(np.ceil(smoothness * len(x)))
    tau = np.sort(DSquare)[position]
    # weight
    w = np.exp(-DSquare/(2*tau*tau))
    # set small weights to 0 to speed up the computation
    w[w<1e-2] = 0 
    return w


# export fixed wavelength and flux 
def _prepareSample(data, name, folder, smoothness=1e-8, resolution=1000):
    
    if (smoothness < 1.0e-2):
        name = name + '_rigid'
    else:
        name = name + '_smooth'
        
    # compute the prediction
    ## observed data
    x = np.array([[1,xi] for xi in data['lam']])
    y = np.array(data['flux'])

    ## to be predicted x
    xPredict = np.linspace(min(data['lam']), max(data['lam']), num=resolution)
    xs = np.array([[1,xi] for xi in xPredict])

    ## run LWR and compute the prediction of y
    thetas = [normalEquation(x,y,smoothness,xsi) for xsi in xs] 
    yPredict=[np.dot(xsi,thetasi) for xsi,thetasi in zip(xs,thetas)]


    # plot
    fig = plt.figure(figsize=(8,6))
    plt.plot(data['lam'],data['flux'],'.')
    plt.plot(xPredict,yPredict,'.-',markersize=3)

    plt.xlabel(r'$\mathrm{Wavelength \ \ [\AA]}$',fontsize=16)
    plt.ylabel(r'$F_{\lambda}  \ \ [10^{-17} \rm{erg/s/cm^{2}/\AA}]  $',fontsize=16) 
    fig.tight_layout()
    plt.savefig(folder + '/' + str(name)+'.pdf')
    plt.close(fig)
    
    
    # convert to dataframe
    result = pd.DataFrame(list(zip(xPredict,yPredict)), columns=['x','y'])
    
    # normalize data
    std = np.std(result['y'])
    avg = np.mean(result['y'])
    eps = 0.001
    yNormalized = (result['y'] - avg)/std
    yNormalized -= np.min(yNormalized)
    yNormalized += eps
    yNormalized /= np.max(yNormalized)
    result['yNormalized'] = yNormalized
    
    # export data
    result.to_csv(folder + '/' + str(name)+'.csv', index=False)
    #return result


In [15]:
data=_getData('./Sy19_train/spec-0266-51630-0227.fits')
_prepareSample(data, 'spec-0266-51630-0227','./Sy19_train')

min_lam = 3824.7241382490856   max_lam= 9185.439827473989


In [6]:
#dir_list=['./Sy19_test','./Sy19_train','./Sy2_test','./Sy2_train']
#dir_list=['./Sy2_train','./Sy2_test']
dir_list=['./Sy2_test']
for i in dir_list:
    file_path = [x for x in os.listdir(i) if x.endswith(".fits")]
    file_name = [y.replace('.fits','') for y in file_path ]
    for j in range(len(file_path)):
        data=_getData(i+'/'+file_path[j])
        _prepareSample(data, file_name[j],i)
        #print(i+'/'+file_path[j])
    

KeyError: 'lam'

In [12]:
for i in dir_list:
    file_path = [x for x in os.listdir(i) if x.endswith(".fits")]
    print(i)
    print(len(file_path))

./Sy19_test
445
./Sy19_train
300
./Sy2_test
53494
./Sy2_train
1200


In [153]:
a=[x for x in os.listdir('./Sy2_test/') if x.endswith(".fits")]
print(len(a))
b=[x for x in os.listdir('./Sy2_test/') if x.endswith(".csv")]
print(len(b))

53494
53494
